In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

C:\Users\zhang\AppData\Local\Temp\ipykernel_19308\1867929233.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# hyper params
batch_size = 100
img_height = 300
img_width = 300

In [3]:
# import the trained model and parameters
trained_model = tf.keras.models.load_model("AccidentDetectionModel.h5")

In [ ]:
acc_vector = []

n = 0
for images, labels in testing_ds.take(1):
    predictions = trained_model.predict(images)
    predict_labels = []  # class_names
    prdind = []  # index
    
    for p in predictions:
        predict_labels.append(class_names[np.argmax(p)])
        prdind.append(np.argmax(p))
    
    acc_vector = (np.array(prdlbl) == labels)
    print('accuracy: ', np.mean(acc_vector))

    if n==0:
        print('predictions:', predictions)
        print('predict_labels:', predict_labels)
        print('prdind:', prdind)
    
    n+=1

In [6]:
def predict_frame(img, model):
    img_array = tf.keras.utils.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    prediction=(model.predict(img_batch) < 0.5).astype("int32")
    print("prediction in predict_frame", prediction)
    if(prediction[0][0]==0):
        return(("Accident Detected", 1))
    else:
        return(("No Accident", 0))

In [ ]:
def run_detection(image_path):
    # 这里应替换为实际的调用逻辑
    # 例如: 使用 subprocess.run([...], capture_output=True) 调用 detect.py 并解析输出
    pass

In [24]:
import subprocess
import re

def run_detection(image_path):
    command = f"python ./yolov5-master/detect.py --source {image_path}"
    result = subprocess.run(command, capture_output=True, text=True, shell=True)
    print("result: ",result)
    output = result.stdout
    if "person" in output:
        print("Person detected in the image.")
        return True
    else:
        print("No person detected in the image.")
        return False


In [25]:
run_detection("./yolov5-master/data/images/zidane.jpg")

result:  CompletedProcess(args='python ./yolov5-master/detect.py --source ./yolov5-master/data/images/zidane.jpg', returncode=1, stdout='', stderr='Traceback (most recent call last):\n  File "C:\\Users\\zhang\\Desktop\\projects\\MSE806ITS\\yolov5-master\\detect.py", line 46, in <module>\n    from ultralytics.utils.plotting import Annotator, colors, save_one_box\nModuleNotFoundError: No module named \'ultralytics\'\n')
No person detected in the image.


False

In [11]:
# read the video stream
images=[]  # list of image frames
pred_labels=[]  # [("accident detected", 1)] list of labels of each image frame, if collision detected
human_detect_labels = []  # 

if_accident_detected = False
sec_count = 0  #  count seconds after collision
alert_threshold = 3  # alert after 3 seconds

cap= cv2.VideoCapture('data_collision/videoplayback_demo.mp4')
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()
fps = cap.get(cv2.CAP_PROP_FPS)  # get the frames per second (fps) of the video stream
frame_counts = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # total frame counts of the video stream
print("fps, frame_counts: ", fps, frame_counts)


for i in range(frame_counts):
    ret, frame = cap.read()  # ret: if frame is successfully grabbed; frame_shape = (360.640,3)
    if not ret:
        print("Error: ret is false.")
        break
    if i%fps==0:
        print(i)
        # print("ret, frame: ", ret, frame, frame.shape)
        # resize the image frame from (360,640) to (300,300)
        resized_frame=tf.keras.preprocessing.image.smart_resize(frame, (img_height, img_width), interpolation='bilinear')
        images.append(frame)
        collision_detect = predict_frame(resized_frame, trained_model)
        pred_labels.append(collision_detect)
        # accident detected == True
        if collision_detect[1]:  
            print("Accident Detected at ", i/fps, " seconds")
            if_accident_detected = True
            # save the frame per second for object detection
            with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmp_file:
                cv2.imwrite(tmp_file.name, frame)
                detected_labels = run_detection(tmp_file.name)
        if if_accident_detected:
            sec_count+=1

cap.release()

NameError: name 'false' is not defined

In [ ]:
import subprocess
import json
from pathlib import Path
import tempfile

# 假设 run_detection 函数是 detect.py 修改后提供的，
# 它接受图像路径作为输入，返回检测到的对象标签列表。
# 如果 detect.py 不支持直接调用，请使用 subprocess 调用命令行并解析结果。
def run_detection(image_path):
    # 这里应替换为实际的调用逻辑
    # 例如: 使用 subprocess.run([...], capture_output=True) 调用 detect.py 并解析输出
    pass

# 修改后的视频处理和检测逻辑
cap = cv2.VideoCapture('data_collision/videoplayback_demo.mp4')
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)  # 获取视频流的帧率
frame_counts = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 获取视频流的总帧数
accident_detected = False
frames_after_accident = 0
alert_threshold = 3 * fps  # 3秒内的帧数

print("fps, frame_counts: ", fps, frame_counts)

for i in range(frame_counts):
    ret, frame = cap.read()
    if not ret:
        break
    
    if i % fps == 0:  # 每秒处理一帧
        # 假设的事故检测逻辑，collision_detect 应为布尔值，表示是否检测到事故
        collision_detect = predict_frame(frame, trained_model)
        
        if collision_detect and not accident_detected:
            print("Accident detected at frame", i)
            accident_detected = True
            frames_after_accident = 0
        
        if accident_detected:
            frames_after_accident += fps
            if frames_after_accident <= alert_threshold:
                # 保存当前帧到临时文件以进行检测
                with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmp_file:
                    cv2.imwrite(tmp_file.name, frame)
                    detected_labels = run_detection(tmp_file.name)
                
                # 检查是否未检测到“人”
                if "person" not in detected_labels:
                    print("Alert: No person detected within 3 seconds after accident.")
                    # 发出警报的逻辑
                    break  # 根据需要决定是否继续检查更多帧或中断循环

cap.release()
